## Imports


In [1]:
from collections import Counter
import re

from nltk import word_tokenize
from nltk.corpus import stopwords
import numpy as np

## Question 1A

Reads a script file (friends_pilot.txt)


In [2]:
# Helper function also used in Programming Quiz 5
def tokenize_text(text):
    return np.array(word_tokenize(text))

In [3]:
# Read the text file
with open("given_files/friends_pilot.txt", "r") as f:
    text = f.read()

# Tokenize it
tokens = tokenize_text(text)

## Question 1B

Removes non-words and stopwords
Suggested: monica paul chandler joey phoebe ross rachel scene n't 's 're 'm na ca gon 'll


In [4]:
# Helper function also used in Programming Quiz 5
def remove_non_words(word_tokens):
    tokens_without_non_words = [
        word for word in word_tokens if re.search(r"[a-zA-Z]", word)
    ]
    return np.array(tokens_without_non_words)


# Helper function also used in Programming Quiz 5
def remove_stop_words(tokens_without_non_words, stop_words):
    tokens_without_stop_words = []
    for word in tokens_without_non_words:
        if word in stop_words or word.lower() in stop_words:
            continue

        tokens_without_stop_words.append(word)
    return np.array(tokens_without_stop_words)


In [5]:
# Remove non-words
tokens_without_nonwords = remove_non_words(tokens)

stop_words = stopwords.words("english")

# Additional words to remove identified in assignment description and other weird ones
additional_words = [
    "Monica",
    "Chandler",
    "Joey",
    "Richard",
    "Phoebe",
    "Ross",
    "Rachel",
    "Thompson",
    "Elizabeth",
    "Paul",
    "'ll",
    "n't",
    "'s",
    "'re",
    "'m",
    "na",
    "gon",
    "ca",
    "I",
    "[",
    "]",
    "Scene",
]

# Add additional words to stopwords
stop_words = stop_words + additional_words

tokens_without_non_and_stop_words = remove_stop_words(
    tokens_without_nonwords, stop_words
)

## Question 2

Create a BoW with the top 20 words


In [6]:
# Helper function to generate BoW
def generate_BoW(tokens):
    counts = Counter(tokens)

    # Sort them based on counts high to low
    sorted_word_counts = sorted(counts.items(), key=lambda word: word[1], reverse=True)

    return sorted_word_counts[:20]

In [7]:
BoW = generate_BoW(tokens_without_non_and_stop_words)

print(BoW)

[('Oh', 29), ('know', 28), ('like', 21), ('got', 17), ('cut', 16), ('get', 15), ('right', 13), ('go', 13), ('Okay', 12), ('God', 11), ('Well', 11), ('think', 11), ('want', 10), ('date', 9), ('back', 9), ('look', 9), ('okay', 9), ('coffee', 9), ('mean', 9), ('Yeah', 9)]


## Question 3

- Generate a Huffman tree and assign codes to them
- Prints out the codes in a nice format in the descending order of their frequencies


In [8]:
# This code was taken from "rec_nov20_2023" and
# https://www.programiz.com/dsa/huffman-coding
# It is slightly modified so that it works with python and numpy strings
class TreeNode(object):
    def __init__(self, left=None, right=None):
        self.left = left
        self.right = right

    def children(self):
        return (self.left, self.right)

    def nodes(self):
        return (self.left, self.right)


# Function to generate the nodes from the Bag of Words
def generate_nodes(BoW):
    BoW_copy = list(BoW)
    while len(BoW_copy) > 1:
        key1, c1 = BoW_copy.pop()
        key2, c2 = BoW_copy.pop()

        node = TreeNode(key1, key2)
        BoW_copy.append((node, c1 + c2))

        BoW_copy = sorted(BoW_copy, key=lambda x: x[1], reverse=True)
    return BoW_copy


# Main function implementing huffman coding
def huffman_code_tree(node, left=True, binString=""):
    # Play nice with numpy
    if type(node) in [str, np.str_]:
        return {node: binString}

    (l, r) = node.children()
    d = dict()
    d.update(huffman_code_tree(l, True, binString + "0"))
    d.update(huffman_code_tree(r, False, binString + "1"))
    return d

In [9]:
nodes = generate_nodes(BoW)

huffman_code = huffman_code_tree(nodes[0][0])

print("Word       Bits         Freq")
print("----------------------------")
for word, frequency in BoW:
    bits = huffman_code[word]
    print(f"{word:8}   {bits:10}   {frequency:4}")

Word       Bits         Freq
----------------------------
Oh         010            29
know       001            28
like       1101           21
got        1000           17
cut        0111           16
get        0110           15
right      0001           13
go         0000           13
Okay       11111          12
God        11110          11
Well       11101          11
think      11100          11
want       11001          10
date       11000           9
back       10011           9
look       10010           9
okay       10101           9
coffee     10100           9
mean       10111           9
Yeah       10110           9
